<a href="https://colab.research.google.com/github/VaishnaviTapetla/HDS5210_Instructor/blob/main/midterm/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [31]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [32]:
def calculate_bmi(height_m, weight_kg):
    """
    Calculate BMI given height in meters and weight in kilograms.

    Args:
    height_m (float): Height in meters
    weight_kg (float): Weight in kilograms

    Returns:
    float: BMI value

    >>> calculate_bmi(1.75, 70)
    22.86
    >>> calculate_bmi(1.60, 55)
    21.48
    >>> calculate_bmi(0, 70)
    Traceback (most recent call last):
    ...
    ValueError: Height and weight must be positive numbers
    """
    if height_m <= 0 or weight_kg <= 0:
        raise ValueError("Height and weight must be positive numbers")
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 2)

### Step 2: Calculate BODE Score

In [33]:
def calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters):
    """
    Calculate BODE score based on the given parameters.

    Args:
    bmi (float): Body Mass Index
    fev_pct (float): FEV1% predicted
    dyspnea_description (str): Description of dyspnea
    distance_in_meters (float): 6-minute walk distance in meters

    Returns:
    int: BODE score (0-10)

    >>> calculate_bode_score(21, 65, "STOPS AFTER 100 YARDS", 350)
    3
    >>> calculate_bode_score(19, 35, "BREATHLESS WHEN DRESSING", 200)
    7
    >>> calculate_bode_score(22, 80, "ONLY STRENUOUS EXERCISE", 450)
    0
    """
    score = 0

    # BMI score
    if bmi < 21:
        score += 1

    # FEV1% predicted score
    if fev_pct >= 65:
        score += 0
    elif 50 <= fev_pct < 65:
        score += 1
    elif 36 <= fev_pct < 50:
        score += 2
    else:
        score += 3

    # MMRC Dyspnea Scale score
    dyspnea_scale = {
        "ONLY STRENUOUS EXERCISE": 0,
        "WHEN HURRYING": 1,
        "WALKING UPHILL": 1,
        "SLOWER THAN PEERS": 2,
        "STOPS AFTER 100 YARDS": 2,
        "STOPS AFTER A FEW MINUTES": 3,
        "STOPS WHEN WALKING AT PACE": 3,
        "BREATHLESS WHEN DRESSING": 4,
        "UNABLE TO LEAVE HOME": 4
    }
    score += dyspnea_scale.get(dyspnea_description.upper(), 0)

    # 6-minute walk distance score
    if distance_in_meters >= 350:
        score += 0
    elif 250 <= distance_in_meters < 350:
        score += 1
    elif 150 <= distance_in_meters < 250:
        score += 2
    else:
        score += 3

    return score


### Step 3: Calculate BODE Risk

In [34]:
def calculate_bode_risk(bode_score):
    """
    Calculate 4-year survival rate based on BODE score.

    Args:
    bode_score (int): BODE score (0-10)

    Returns:
    float: 4-year survival rate percentage

    >>> calculate_bode_risk(0)
    0.82
    >>> calculate_bode_risk(5)
    0.57
    >>> calculate_bode_risk(10)
    0.18
    """
    if bode_score < 0 or bode_score > 10:
        raise ValueError("BODE score must be between 0 and 10")

    survival_rates = {
        0: 0.82, 1: 0.82, 2: 0.82,
        3: 0.68, 4: 0.68,
        5: 0.57, 6: 0.57,
        7: 0.42, 8: 0.42,
        9: 0.18, 10: 0.18
    }

    return survival_rates[bode_score]



### Step 4: Load Hospital Data

In [35]:
def load_hospital_data(file_path):
    """
    Load hospital data from a JSON file.

    Args:
    file_path (str): Path to the JSON file

    Returns:
    dict: Hospital data with hospital names as keys and bed counts as values
    """
    with open(file_path, 'r') as f:
        data = json.load(f)

    hospital_beds = {}
    for system in data:
        for hospital in system['hospitals']:
            hospital_beds[hospital['name']] = hospital['beds']

    return hospital_beds

### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [36]:
def process_patient_data(patient_csv, hospital_json, patient_output_file, hospital_output_file):
    hospital_beds = load_hospital_data(hospital_json)
    hospital_data = {hospital: {'count': 0, 'total_score': 0, 'total_risk': 0} for hospital in hospital_beds}

    patient_results = []

    with open(patient_csv, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row['NAME']
            height_m = float(row['HEIGHT_M'])
            weight_kg = float(row['WEIGHT_KG'])
            fev_pct = float(row['fev_pct'])
            dyspnea_description = row['dyspnea_description']
            distance_in_meters = float(row['distance_in_meters'])
            hospital = row['hospital']

            bmi = calculate_bmi(height_m, weight_kg)
            bode_score = calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters)
            bode_risk = calculate_bode_risk(bode_score)

            patient_results.append([name, bode_score, bode_risk, hospital])

            hospital_data[hospital]['count'] += 1
            hospital_data[hospital]['total_score'] += bode_score
            hospital_data[hospital]['total_risk'] += bode_risk

    # Write patient_output.csv
    with open(patient_output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['NAME', 'BODE_SCORE', 'BODE_RISK', 'HOSPITAL'])
        writer.writerows(patient_results)

    # Prepare and write hospital_output.csv
    hospital_output_list = []
    for hospital, data in hospital_data.items():
        count = data['count']
        if count > 0:
            avg_score = data['total_score'] / count
            avg_risk = data['total_risk'] / count
            pct_cases_over_beds = (count / hospital_beds[hospital]) * 100 if hospital_beds[hospital] > 0 else 0
            hospital_output_list.append([
                hospital,
                count,
                round(pct_cases_over_beds, 2),
                round(avg_score, 2),
                round(avg_risk, 2)
            ])

    with open(hospital_output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['HOSPITAL_NAME', 'COPD_COUNT', 'PCT_OF_COPD_CASES_OVER_BEDS', 'AVG_SCORE', 'AVG_RISK'])
        writer.writerows(hospital_output_list)

# Run doctests
if __name__ == "__main__":
    doctest.testmod()

    # Process the data
    patient_csv = "patient.csv"
    hospital_json = "hospitals.json"
    patient_output_file = "patient_output.csv"
    hospital_output_file = "hospital_output.csv"

    process_patient_data(patient_csv, hospital_json, patient_output_file, hospital_output_file)

**********************************************************************
File "__main__", line 14, in __main__.calculate_bode_score
Failed example:
    calculate_bode_score(21, 65, "STOPS AFTER 100 YARDS", 350)
Expected:
    3
Got:
    2
**********************************************************************
File "__main__", line 16, in __main__.calculate_bode_score
Failed example:
    calculate_bode_score(19, 35, "BREATHLESS WHEN DRESSING", 200)
Expected:
    7
Got:
    10
**********************************************************************
1 items had failures:
   2 of   3 in __main__.calculate_bode_score
***Test Failed*** 2 failures.
